In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from fastai.vision import *

In [ ]:
df = pd.read_csv("../input/train_labels.csv")
df.head()

In [ ]:
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)

In [ ]:
np.random.seed(42)
src = (ImageItemList.from_csv('../input', 'train_labels.csv', folder='train', suffix='.tif')
       .random_split_by_pct(0.2)
       .label_from_df(label_delim=' '))

In [ ]:
data = (src.transform(tfms, size=128)
        .databunch().normalize(imagenet_stats))

In [ ]:
data.show_batch(rows=3, figsize=(12,9))

In [ ]:
arch = models.resnet50

In [ ]:
df['label'].value_counts()

In [ ]:
PATH = "../input/"
print("training data")
!ls -l {PATH}train | grep ^[^dt] | wc -l

In [ ]:
print("test data")
!ls -l {PATH}test | grep ^[^dt] | wc -l

**All code below this cell are Work In Progress [WIP]**

In [ ]:
from os.path import *
cache_dir = expanduser(join('~', '.torch'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [ ]:
!ls ~/.torch/models

In [ ]:
import pathlib
arch.path = pathlib.Path('.')

In [ ]:
!cp ../input/models/resnet50-19c8e357.pth ~/.torch/models/resnet50-19c8e357.pth

In [ ]:
learn = create_cnn(data,arch, path='.')

In [ ]:
create_cnn??


In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr = 2.5e-3
learn.fit_one_cycle(5,slice(lr))

In [ ]:
learn.save('stage-1-rn50')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()


In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5,slice(5e-6,lr/5))

In [ ]:
learn.save('stage-2-rn50')

In [ ]:
data = (src.transform(tfms, size=256)
        .databunch().normalize(imagenet_stats))

learn.data = data
data.train_ds[0][0].shape

In [ ]:
learn.freeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()